In [ ]:
import pandas as pd
import json
import datetime
from azure.storage.blob import BlobClient
from io import BytesIO


feet_per_furlong = 660
with open('local.settings.json','r') as f:
    settings = json.load(f)
    f.close()
   
cx = settings["AzureWebJobsStorage"]

#create stat grouping on jockey + trainer
#get average time for distance raced at track and compare to average time across all tracks
#get speed of horse at varying distances
#determine metric based on race category (maiden, claiming, etc)
#determine metric based on lasix use
#determin metric based on horse age
#determine metric based on odds - done
#determine metric based on track condition
#determine metric based on track type
#determine metric based on track surface
#determine metric based on starting position - done

In [ ]:
def download_file(input_container,blob_name, connection_string):
    cx = connection_string
    bc = BlobClient.from_connection_string(conn_str=cx, container_name=input_container, blob_name=blob_name)
    stream = BytesIO()
    data = bc.download_blob()
    data.download_to_stream(stream)
    stream.seek(0)
    ret_val = pd.read_csv(stream,low_memory=False)
    ret_val['track_last_raced'].fillna('', inplace=True)
    return ret_val

In [ ]:
def get_track_par(df,track,race_date,race_type,surface,distance):
    df = df.query("date<'{}'".format(race_date))
    query = "track=='{}' and distance_float=={} and position==1.0".format(track,distance)

    #print(query)
    track_history = df.query(query)
    #print('Track History')
    #display(track_history.sort_values(by=['date']))
            

    track_history = track_history.dropna(axis='columns',how='all')
    #print('DF SIZE: {}'.format(len(track_history.index)))

    if track_history.empty == True or len(track_history.index) < 5:
        #print('Small data size: {}'.format(len(track_history.index)))
        query = "type=='{}' and distance_float=={} and position==1.0".format(race_type,distance)
        track_history = df.query(query)
        track_history = track_history.describe()
    else:
        track_history = track_history.describe()
    #display(track_history)
    return track_history

In [ ]:
def setup_track_par(df, entry,  isEmpty=False):
    horse_name = entry['name']
    race_type = entry['type']
    #race_type_level = entry['claim_price']
    surface = entry['surface']
    distance = entry['distance_float']
    prev_track = entry['track']
    race_date = entry['date']
    
    # curr_race_type = the_race.iloc[0]['type']
    # #race_type_level = entry['claim_price']
    # curr_surface = the_race.iloc[0]['surface']
    # curr_distance = the_race.iloc[0]['distance_float']
    # curr_race_date = the_race.iloc[0]['date']
    # curr_track = the_race.iloc[0]['track']

    axis = '50%'
    if isEmpty is True:
        axis = '25%'

    prev_track_stats = get_track_par(df,prev_track,race_date,race_type,surface,distance)
    #prev_track_diff = prev_track_stats.loc['50%', 'ind_fin'] / entry['ind_fin']

    #curr_track_stats = get_track_par(df,curr_track,curr_race_date,curr_race_type,curr_surface,curr_distance)
    #curr_track_diff = prev_track_stats.loc['50%', 'ind_fin'] / entry['ind_fin']

    # if prev_track_stats.empty:
    #     prev_track_stats = curr_track_stats

    #display(prev_track_stats)
    #display(curr_track_stats)


    row_to_add = {
        'horse_name' : horse_name,
        'date' : race_date,
        'prev_track' : prev_track,
        'horse_time' : entry['ind_fin'],
        'track_par_med' : prev_track_stats.loc[axis, 'ind_fin'],
        'adj_diff' : prev_track_stats.loc['50%', 'ind_fin'] - prev_track_stats.loc['mean', 'ind_fin'],
        'difference_med' : prev_track_stats.loc['50%', 'ind_fin'] / entry['ind_fin'],
        #'curr_track' : curr_track,
        #'curr_track_par' : curr_track_stats.loc['50%', 'ind_fin'],
        #'track_par_diff' : curr_track_diff,       
        'distance' : distance,
    }

    return row_to_add

In [ ]:
def speed_figure(horse_list,df, str_end_date):

    #get unique tracks that horses participated in
    #prev_raced_tracks = the_race['track_last_raced'].unique().tolist()
    date = str_end_date
    #distance = the_race.iloc[0]['distance_float']

    total_stack_up = pd.DataFrame()
    horse_summary = pd.DataFrame()

    count = 0
    total = len(horse_list)
    for horse in horse_list:
        count+=1
        #print('{}/{} | {}'.format(count,total, horse))
        trend = 0
        predicted_value = 0
        stack_up = pd.DataFrame()
        #get race history
        horse_name = horse
        #print(horse_name)
        horse_history_query = 'name=="{}" and date < "{}"'.format(horse_name,date)   
        horse_history = df.query(horse_history_query)
        horse_history = horse_history.sort_values(by=['date'])
        horse_history = horse_history.dropna(axis='columns',how='all')

        
       # display(horse_history_short)
        #prev_raced_tracks = horse_history['track_last_raced'].unique().tolist()
        #breeds_raced = horse_history['breed'].unique().tolist()

        if horse_history.empty or len(horse_history.index) == 0:
            summary_data = {
                'horse_name' : horse_name,
                'speed_to_par' : 0,
                'trend' : trend,
                'predicted_to_par' : predicted_value
            }
            horse_summary = pd.concat([horse_summary, pd.DataFrame([summary_data])], ignore_index=True)
            continue
            #print('{} has no history'.format(horse_name))
            # row_to_add = setup_track_par(df,entry, isEmpty=True)
            # stack_up = stack_up.append(row_to_add, ignore_index=True)
            # total_stack_up = total_stack_up.append(row_to_add, ignore_index=True)
            # predicted_value = stack_up.describe().loc['50%', 'difference_med'] 
        
        else:
            # try:
            #     horse_history_short = horse_history[['name','distance_float','seconds','type', 'choice', 'position']]
            # except:
            #     horse_history_short = horse_history[['name','distance_float','seconds','type', 'position']]
            race_times = []
            for rdx, race in horse_history.iterrows():
                #if race['distance_float'] >= (distance-1.5) and race['distance_float'] <= (distance+1.5):
                try:
                    if True:
                        row_to_add = {}#setup_track_par(df,race)
                        row_to_add['name'] = race['name']
                        row_to_add['distance_float'] = race['distance_float']
                        row_to_add['seconds'] = race['seconds']
                        row_to_add['type'] = race['type']
                        row_to_add['position'] = race['position']
                        row_to_add['feet_per_second'] = race['seconds'] / race['distance_float']
                        row_to_add['readiness'] = 0
                        try:
                            row_to_add['choice'] = race['choice']
                        except:
                            row_to_add['choice'] = 0
                        stack_up = pd.concat([stack_up, pd.DataFrame([row_to_add])], ignore_index=True)
                        total_stack_up = pd.concat([total_stack_up, pd.DataFrame([row_to_add])], ignore_index=True)
                except:
                    continue
    return total_stack_up

    #         #stack_up = stack_up.sort_values(by=['date'])
    #         #try:
    #         if stack_up.empty or len(stack_up.index) == 0:
    #             summary_data = {
    #                 'horse_name' : horse_name,
    #                 'speed_to_par' : 0,
    #                 'trend' : trend,
    #                 'predicted_to_par' : predicted_value
    #             }
                
    #             horse_summary = pd.concat([horse_summary, pd.DataFrame([summary_data])], ignore_index=True)
    #             continue

            
    #         stack_up = stack_up.fillna(0)
    #         par_trend = stack_up['difference_med'].to_list()
            
    #         std_dev_val = stack_up.describe().loc['std','difference_med']
    #         med_val = stack_up.describe().loc['50%','difference_med']
    #         normalized_val = []
    #         #remove outliers
    #         #print(std_dev_val)
    #         #print(med_val)
    #         #for x in range(0,len(par_trend)-1):
    #         for x in par_trend:
    #             val = x#par_trend[x]
    #             limit_up = med_val + std_dev_val
    #             limit_down = med_val - std_dev_val
    #             #print('{} > {} or {} < {}'.format(val,limit_up,val,limit_down))
    #             if val > limit_up or val < limit_down:
    #                 #print('popping {}'.format(val))
    #                 continue
    #                 #par_trend.pop(x)
    #             else:
    #                 #print('adding {}'.format(val))   
    #                 normalized_val.append(val)
                    
    #         # except Exception as e:
    #         #     print(e)
    #         #     print(normalized_val)
    #         #     print('par trend may be empty: {}'.format(par_trend))
    #         #print(par_trend)
    #         if len(normalized_val) > 0:
    #             for val in range(0,len(normalized_val)-2):
    #                 current = normalized_val[val]
    #                 next_itr = normalized_val[val+1]
    #                 trend = trend + (current - next_itr)

    #             horse_stats = stack_up.describe()
    #             #display(horse_stats)
    #             predicted_value = normalized_val[-1] - trend
    #             axis_val = 'mean'

    #             #if trend < 0 use 50% and 75%
    #             #if trend > 0 use mean and 25%
    #             if trend > 0 and trend <= std_dev_val :
    #                 axis_val = 'mean'
    #                 last_time = normalized_val[-1]
    #                 min_time = horse_stats.loc['min','difference_med']
    #                 mean_time = horse_stats.loc[axis_val,'difference_med']
    #                 lower_qtr_time = horse_stats.loc['25%', 'difference_med']
                    
    #                 if last_time < mean_time:
    #                     axis_val = '25%'
    #                 if last_time < lower_qtr_time:
    #                     axis_val = 'min'

                
    #             elif trend > (std_dev_val*1) and trend <= (std_dev_val*2):
    #                 axis_val = '25%'
    #             elif trend > (std_dev_val*2):
    #                 axis_val = 'min'

    #             elif trend < 0 and trend >= (std_dev_val*-1):
    #                 axis_val = '50%'
    #             elif trend < (std_dev_val*-1) and trend >= (std_dev_val*-1):
    #                 axis_val = '75%'
    #                 last_time = normalized_val[-1]
    #                 min_time = horse_stats.loc['min','difference_med']
    #                 lower_qtr_time = horse_stats.loc[axis_val,'difference_med']
                    
    #                 if last_time < lower_qtr_time:
    #                     axis_val = 'max'
    #             elif trend < (std_dev_val*-2):
    #                 axis_val = 'max'

    #             predicted_value = horse_stats.loc[axis_val, 'difference_med']
    #         else:
    #             trend = 0
    #             predicted_value = stack_up.describe().loc['50%', 'difference_med']

    #     summary_data = {
    #         'horse_name' : horse_name,
    #         'speed_to_par' : stack_up.describe().loc['50%', 'difference_med'] ,
    #         'trend' : trend,
    #         'predicted_to_par' : predicted_value
    #     }
    #     #display(stack_up)
    #     horse_summary = pd.concat([horse_summary, pd.DataFrame([summary_data])], ignore_index=True)
            

    # column_order = ['date', 'horse_name', 'horse_time', 'distance','prev_track', 'track_par_med', 'adj_diff','difference_med','curr_track', 'curr_track_par']
    # total_stack_up = total_stack_up.reindex(columns=column_order)
    # #display(total_stack_up)
    

    # return horse_summary.sort_values(by=['speed_to_par'], ascending=False)

In [ ]:
def get_post_winning_perc(the_race, df):
    
    post_winning_perc = pd.DataFrame()
    df_subset = df.query('track=="{}" and date < "{}" and pos_fin<5'.format(the_race.iloc[0]['track'], the_race.iloc[0]['date']))
    #unique_races = post_query.drop_duplicates(['date', 'track', 'race_number'])and distance_float=={}  the_race.iloc[0]['distance_float'] 
    race_count = len(df_subset.index)

    for p, racer in the_race.iterrows():
        post_data = {
            'horse_name' : '',
            'post_perc' : 0
        }
        post_query = df_subset.query('pp=={}'.format(racer['pp']))
        post_winners = len(post_query.index)
        post_data['pp'] = racer['pp']
        post_data['horse_name'] = racer['name']
        post_data['post_perc'] = post_winners / race_count
        post_winning_perc = pd.concat([post_winning_perc, pd.DataFrame([post_data])], ignore_index=True)
    return post_winning_perc.sort_values(by=['post_perc'], ascending=False)


In [ ]:
def lowest_odds_calc(the_race, winning_perc):
    the_race = the_race.sort_values(by=['odds'], ascending=True)
    lowest_odds = the_race.iloc[0]['odds']

    odds_calc = pd.DataFrame()
    for o, row in the_race.iterrows():
        data = {
            'horse_name': '',
            'odds' : 0
        }

        data['horse_name'] = row['name']
        data['odds'] = (lowest_odds / row['odds']) * winning_perc
        odds_calc = pd.concat([odds_calc, pd.DataFrame([data])], ignore_index=True)
        

    return odds_calc.sort_values(by=['odds'], ascending=False)

In [ ]:
def horse_favorite_winning_percentage(df, the_race):
    unique_races = df.drop_duplicates(['date', 'track', 'race_number'])
    race_count = len(unique_races.index)

    odds_place_perc_df = pd.DataFrame()
    
    for c, crow in the_race.iterrows():
        place = 0
        data = {}
        data['horse_name'] = crow['name']
        while place <= len(the_race.index):
            try:
                place += 1
                odds_place_rank = 'odd_place_rank_{}'.format(place)
                fav_perc = df.query('pos_fin<={} and choice=={}'.format(place,crow['choice']))  
                odds_place_count = len(fav_perc.index)
                odds_place_perc = odds_place_count / race_count
                data['choice'] = crow['choice']
                data[odds_place_rank] = odds_place_perc
            except:
                data['horse_name'] = crow['name']
                data[odds_place_rank] = .1
        odds_place_perc_df = pd.concat([odds_place_perc_df, pd.DataFrame([data])], ignore_index=True)

    return odds_place_perc_df

In [ ]:
def trainer_with_jockey(df, the_race):
    trainer_n_jockey = pd.DataFrame()
    
    for x, xtj in the_race.iterrows():
        df_subset = df.query('trainer=="{}" and jockey=="{}" and date < "{}"'.format(xtj['trainer'], xtj['jockey'], xtj['date']))
        
        num_races = len(df_subset.index)
        qual_finish_df = df_subset.query('pos_fin<5')
        qual_finish_num = len(qual_finish_df.index)
        if num_races > 0:
            usable_races = {}
            usable_races['trainer'] = xtj['trainer']
            usable_races['jockey'] = xtj['jockey']
            usable_races['scoring_perc'] =  qual_finish_num / num_races
            trainer_n_jockey = pd.concat([trainer_n_jockey, pd.DataFrame([usable_races])], ignore_index=True)
    
    return trainer_n_jockey.sort_values(by=['scoring_perc'], ascending=False)
    
    

In [ ]:
def predict_horse_race(the_race, full_race_data, str_end_date):
    pd.set_option('display.max_columns', None)
    predict = the_race[['name','choice','odds','type','days_since','jockey','trainer','trainer_rank','jockey_rank']]
    low_odds_win_perc_one = horse_favorite_winning_percentage(full_race_data, the_race)
    tj_combine = trainer_with_jockey(full_race_data, the_race)
    scoring_prec = tj_combine['scoring_perc']
    horse_list = the_race['name'].unique().tolist()
    horse_rank = speed_figure(horse_list, full_race_data, str_end_date)

    for h,hrow in predict.iterrows():
        for t,trow in tj_combine.iterrows():
            if hrow['trainer'] == trow['trainer'] and hrow['jockey'] == trow['jockey']:
                predict.loc[h,'scoring_prec'] = trow['scoring_perc']
                break
            else:
                predict.loc[h,'scoring_prec'] = 0.0
        for s,srow in horse_rank.iterrows():
            if hrow['name'] == srow['name']:
                days_since = hrow['days_since']
                readiness = 0
                if predict.loc[h,'type'] == 'ALLOWANCE' or predict.loc[h,'type'] == 'STAKES':
                    readiness = predict.loc[h,'days_since'] / 30
                else:
                    readiness = predict.loc[h,'days_since'] / 14

                predict.loc[h, 'readiness'] = readiness
                predict.loc[h,'feet_per_second'] = srow['feet_per_second']

                break
            else:
                predict.loc[h,'readiness'] = 0.0

    predict['trainer_rank'] = predict['trainer'].rank(method='max', ascending=True) 
    predict['jockey_rank'] = predict['jockey'].rank(method='max', ascending=True)
    predict['readiness_rank'] = predict['readiness'].rank(method='max', ascending=False) 
    predict['speed_rank'] = predict['feet_per_second'].rank(method='max', ascending=False)
    for h, hrow in predict.iterrows():
        predict.loc[h,'final_horse_rank'] = predict.loc[h, 'readiness_rank'] + predict.loc[h, 'speed_rank'] + predict.loc[h, 'choice']
        predict.loc[h,'final_rank'] = predict.loc[h, 'readiness_rank'] + predict.loc[h, 'speed_rank'] + predict.loc[h, 'choice'] + predict.loc[h, 'trainer_rank'] + predict.loc[h, 'jockey_rank']

    predict = predict.sort_values(by=['final_rank'], ascending=True)
    return predict

In [ ]:
def create_summary(initial_rank, the_race, summary):
    
    matches = 0
    race_detail = ''
    for i in range(4):
        for j in range(4):  
            race_detail = f"{the_race.iloc[j]['date']}_{the_race.iloc[j]['track']}_{the_race.iloc[j]['race_number']}"  
            if the_race.iloc[j]['name'] == initial_rank.iloc[i]['name']:
                matches += 1
                break
    
    row = pd.DataFrame({'race_id': race_detail, 'matches': [matches]})
    if matches > 2:
        summary = pd.concat([summary, row], ignore_index=True)
    return summary

In [ ]:
def apply_weights(initial_rank, the_race, summarize_results):
    start = 0
    end = 100
    step = 5
    weighted_test = pd.DataFrame(columns=['weights', 'success'])
    
    for horse in range(start,end,step):
        for jockey in range(start,end,step):
            monitor = horse+jockey
            if monitor > 100:
                break
            for trainer in range(start,end,step):
                monitor = horse+jockey+trainer
                if monitor > 100:
                    break
                if monitor < 100:
                    continue

                
                for h,hrow in initial_rank.iterrows():
                    weighted_horse_rank = initial_rank.loc[h,'final_horse_rank'] * (horse/100)
                    weighted_trainer_rank = initial_rank.loc[h,'trainer_rank'] * (trainer/100)
                    weighted_jockey_rank = initial_rank.loc[h,'jockey_rank'] * (jockey/100)
                    initial_rank.loc[h,'weighted_final_rank'] = weighted_horse_rank + weighted_trainer_rank + weighted_jockey_rank
                initial_rank = initial_rank.sort_values(by=['weighted_final_rank'], ascending=True)
                
                summarize_results = create_summary(initial_rank,the_race,summarize_results)
                last_row = len(summarize_results.index) - 1
                #row = pd.DataFrame(columns=['weights', 'success'])
                summarize_results.at[last_row, 'weights'] = 'Horse: {} Jockey: {} Trainer: {}'.format(horse,jockey,trainer)
                summarize_results = summarize_results.sort_values(by=['matches'], ascending=False)

                #weighted_test = pd.concat([weighted_test, row], ignore_index=True)
    return summarize_results

In [129]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
#read in full data
#full_race_data = download_file(settings['input_container'], 'full_race_data.csv', cx)
#read in jockey data
jockey_race_data = download_file(settings['input_container'], 'full_race_data_jockey.csv', cx)
#read in trainer data
trainer_race_data = download_file(settings['input_container'], 'full_race_data_trainer.csv', cx)
full_race_data = jockey_race_data#pd.concat([jockey_race_data,trainer_race_data],ignore_index=True)
full_race_data['trainer_rank'] = trainer_race_data['trainer_rank']

cname = full_race_data.columns.to_list()  
item = 0
while item < len(cname):
    if cname[item].find('Unnamed') > -1:
        full_race_data.drop(full_race_data.columns[[item]], axis=1, inplace=True)
        cname.pop(item)
        item = -1
    item+=1
current_date = datetime.datetime(year=2021,month=9,day=1)

str_end_date = current_date.strftime('%Y-%m-%d')
post_query = full_race_data.query("date <'{}'".format(str_end_date))
unique_races = post_query.drop_duplicates(['date', 'track', 'race_number'])

races_by_date = full_race_data.query("date=='{}'".format(str_end_date))
tracks_raced = races_by_date['track'].unique()
summarize_results = pd.DataFrame(columns=['race_id', 'matches'])

for track in tracks_raced:
    if track != 'SAR':
        continue
    track_races = races_by_date.query("track=='{}'".format(track)).sort_values(by=['race_number','date'],ascending=True)
    num_races = track_races['race_number'].unique().tolist()
    
    for i in num_races:
        try:
            start_time = datetime.datetime.now()
            the_race = track_races.query("race_number=={}".format(i))
            the_race = the_race.dropna(axis='columns',how='all')
            initial_rank = predict_horse_race(the_race, full_race_data, str_end_date)
            summarize_results = apply_weights(initial_rank, the_race, summarize_results)
            #summarize_results = create_summary(initial_rank, the_race, summarize_results)
            
            #display(the_race)
            #display(initial_rank)
            
        except Exception as e:
            display(e)
            continue

    summarize_results = summarize_results.sort_values(by=['weights'], ascending=[False])
    display(summarize_results)
    #display(weighted_results)
        

        ##################################################
        # display('Horse processing')
        # horse_list = the_race['name'].unique().tolist()
        # horse_rank = speed_figure(horse_list, full_race_data, str_end_date)
        # display(horse_rank)
        
        # display('Post Draw processing')
        # pp_winning_perc = get_post_winning_perc(the_race, full_race_data)
        # display(pp_winning_perc)
        
        # display('Odds processing')
        # low_odds_win_perc_one = horse_favorite_winning_percentage(full_race_data, the_race)
        # display(low_odds_win_perc_one)

,race_id,matches,weights
998,2021-09-01_SAR_9,3,Horse: 95 Jockey: 5 Trainer: 0
124,2021-09-01_SAR_1,3,Horse: 95 Jockey: 5 Trainer: 0
415,2021-09-01_SAR_7,3,Horse: 95 Jockey: 5 Trainer: 0
310,2021-09-01_SAR_2,3,Horse: 95 Jockey: 5 Trainer: 0
431,2021-09-01_SAR_4,3,Horse: 95 Jockey: 0 Trainer: 5
871,2021-09-01_SAR_8,3,Horse: 95 Jockey: 0 Trainer: 5
233,2021-09-01_SAR_1,3,Horse: 95 Jockey: 0 Trainer: 5
981,2021-09-01_SAR_9,3,Horse: 95 Jockey: 0 Trainer: 5
93,2021-09-01_SAR_5,3,Horse: 95 Jockey: 0 Trainer: 5
747,2021-09-01_SAR_7,3,Horse: 95 Jockey: 0 Trainer: 5


In [ ]:
pd.set_option('display.max_columns', None)
display(the_race)
predict = the_race[['name','choice','odds','type','days_since','jockey','trainer','trainer_rank','jockey_rank']]
#
low_odds_win_perc_one = horse_favorite_winning_percentage(full_race_data, the_race)
#display(low_odds_win_perc_one)
#pp_winning_perc = get_post_winning_perc(the_race, full_race_data)
#display(pp_winning_perc)
tj_combine = trainer_with_jockey(full_race_data, the_race)

scoring_prec = tj_combine['scoring_perc']
#display(tj_combine)
horse_list = the_race['name'].unique().tolist()

#find head to head results
horse_rank = speed_figure(horse_list, full_race_data, str_end_date)

for h,hrow in predict.iterrows():
    for t,trow in tj_combine.iterrows():
        if hrow['trainer'] == trow['trainer'] and hrow['jockey'] == trow['jockey']:
            predict.loc[h,'scoring_prec'] = trow['scoring_perc']
            break
        else:
            predict.loc[h,'scoring_prec'] = 0.0
    for s,srow in horse_rank.iterrows():
        if hrow['name'] == srow['name']:
            #predict.loc[h,'speed_figure'] = srow['speed_to_par']
            days_since = hrow['days_since']
            readiness = 0
            if predict.loc[h,'type'] == 'ALLOWANCE' or predict.loc[h,'type'] == 'STAKES':
                readiness = predict.loc[h,'days_since'] / 30
            else:
                readiness = predict.loc[h,'days_since'] / 14
            
            predict.loc[h, 'readiness'] = readiness
            predict.loc[h,'feet_per_second'] = srow['feet_per_second']
            
            break
        else:
            predict.loc[h,'readiness'] = 0.0

            
predict['trainer_rank'] = predict['trainer'].rank(method='max', ascending=True) 
predict['jockey_rank'] = predict['jockey'].rank(method='max', ascending=True)
predict['readiness_rank'] = predict['readiness'].rank(method='max', ascending=False) 
predict['speed_rank'] = predict['feet_per_second'].rank(method='max', ascending=False)
for h, hrow in predict.iterrows():
    predict.loc[h,'final_horse_rank'] = predict.loc[h, 'readiness_rank'] + predict.loc[h, 'speed_rank'] + predict.loc[h, 'choice']
    predict.loc[h,'final_rank'] = predict.loc[h, 'readiness_rank'] + predict.loc[h, 'speed_rank'] + predict.loc[h, 'choice'] + predict.loc[h, 'trainer_rank'] + predict.loc[h, 'jockey_rank']

predict = predict.sort_values(by=['final_rank'], ascending=True)
#display(horse_rank)
#display(low_odds_win_perc_one)
display(predict)